In [1]:
import pandas as pd
import numpy as np

#File to load
file = "Seasons_stats.csv"

#Reading file and storing to dataframe
nba_data = pd.read_csv(file)

nba_data.head()

#Used to display all columns
#for col in nba_data.columns: 
    #print(col)

/Users/scotttheener/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (0,6,7,8,9,10,13,14,15,16,17,18,19,20,22,23,24,25,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Unnamed: 0,Year,Player,Pos,Age,Tm,G,GS,MP,PER,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,Key,NaN,NaN,NaN,NaN,NaN,Games,Games Started,Minutes Played,Player Efficency Rating,...,Free Throw Percentage,offensive rebounds,defensive rebounds,total rebounds,assists,steals,Blocks,Turnovers,Personal Fouls,Points
1,0,1950.0,Curly Armstrong,G-F,31.0,FTW,63,NaN,NaN,NaN,...,0.705,NaN,NaN,NaN,176,NaN,NaN,NaN,217,458
2,1,1950.0,Cliff Barker,SG,29.0,INO,49,NaN,NaN,NaN,...,0.708,NaN,NaN,NaN,109,NaN,NaN,NaN,99,279
3,2,1950.0,Leo Barnhorst,SF,25.0,CHS,67,NaN,NaN,NaN,...,0.698,NaN,NaN,NaN,140,NaN,NaN,NaN,192,438
4,3,1950.0,Ed Bartels,F,24.0,TOT,15,NaN,NaN,NaN,...,0.559,NaN,NaN,NaN,20,NaN,NaN,NaN,29,63


**Adding Minutes/Game Calculation to the CSV**

In [2]:
#Filtering data to specific years and grouping by player
player_data = nba_data.loc[nba_data['Year'] >= 1980]

#Dropping NA/NaN values throughout the CSV
player_data.dropna(how='any')

#Setting the G, MP, and GS columns equal to non-NA/NaN values only
player_data = player_data[player_data['G'].notna()]
player_data = player_data[player_data['MP'].notna()]
player_data = player_data[player_data['GS'].notna()]

#Converting the G, MP columns to numberic dtype
player_data['G'] = pd.to_numeric(player_data['G'])
player_data['MP'] = pd.to_numeric(player_data['MP'])
player_data['FGA'] = pd.to_numeric(player_data['FGA'])

#Making new variables for G, MP
games = player_data['G']
minutes = player_data['MP']
FGA = player_data['FGA']

#Calculation for new column called Minutes/Game
minutes_per_game = minutes/games
FGA_per_game = FGA/games

#Adding Minutes/Game column to the CSV, mapping the column, converting to numeric dtype, applying comparison statement
player_data['Minutes/Game'] = minutes_per_game
player_data['Minutes/Game'] = player_data['Minutes/Game'].map('{:.1f}'.format)
player_data['Minutes/Game'] = pd.to_numeric(player_data['Minutes/Game'])
player_data = player_data.loc[player_data['Minutes/Game'] >= 20]

#Adding FGA/Game column to the dataframe
player_data['FGA/Game'] = FGA_per_game
player_data['FGA/Game'] = player_data['FGA/Game'].map('{:.1f}'.format)
player_data['FGA/Game'] = pd.to_numeric(player_data['FGA/Game'])

player_data.head()

,Unnamed: 0,Year,Player,Pos,Age,Tm,G,GS,MP,PER,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Minutes/Game,FGA/Game
5731,5730,1980.0,Tiny Archibald*,PG,31.0,BOS,80.0,80,2864.0,15.3,...,138,197,671,106,10,242,218,1131,35.8,9.9
5749,5748,1980.0,Larry Bird*,PF,23.0,BOS,82.0,82,2955.0,20.5,...,636,852,370,143,53,263,279,1745,36.0,17.8
5782,5781,1980.0,M.L. Carr,SF,29.0,BOS,82.0,7,1994.0,14.6,...,224,330,156,120,36,143,214,914,24.3,9.3
5803,5802,1980.0,Dave Cowens*,C,31.0,BOS,66.0,55,2159.0,14.8,...,408,534,206,69,61,108,216,940,32.7,14.1
5836,5835,1980.0,Chris Ford,SG,31.0,BOS,73.0,73,2115.0,13,...,104,181,215,111,27,105,178,816,29.0,9.7


**Creating dataframe to determine top ten points in a season and points per minute played**

In [3]:
#Creating a series from the CSV with Year, Player, PTS, and our new column Minutes/Game
player_points = player_data[['Year','Player', 'PTS', 'Minutes/Game']]

#Converting the PTS column to a numberica dtype
player_points['PTS'] = pd.to_numeric(player_points['PTS'])

#Sorting and displaying to check if the Minutes/Game column is calculating correctly
top_points = player_points.sort_values(by='PTS', ascending=False)
top_points.head(10)

/Users/scotttheener/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,Year,Player,PTS,Minutes/Game
8470,1987.0,Michael Jordan*,3041.0,40.0
8870,1988.0,Michael Jordan*,2868.0,40.4
17743,2006.0,Kobe Bryant,2832.0,41.0
9748,1990.0,Michael Jordan*,2753.0,39.0
9298,1989.0,Michael Jordan*,2633.0,40.2
22439,2014.0,Kevin Durant,2593.0,38.5
10213,1991.0,Michael Jordan*,2580.0,37.0
24655,2017.0,Russell Westbrook,2558.0,34.6
6558,1982.0,George Gervin*,2551.0,35.7
11107,1993.0,Michael Jordan*,2541.0,39.3


# Offensive Stat 1 - Points/Minutes Played (sorted by PTS)

In [6]:
#Calculation, creation, mapping of new column called Points/Min
points_per_game = player_points['PTS'] / player_data['G']
player_points['PTS/Game'] = points_per_game
player_points['PTS/Game'] = player_points['PTS/Game'].map('{:.1f}'.format)

#Displaying the final calculation
top_points_min = player_points.sort_values(by='PTS', ascending=False)
top_points_min.head(10)

/Users/scotttheener/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/scotttheener/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,Year,Player,PTS,Minutes/Game,PTS/Game
8470,1987.0,Michael Jordan*,3041.0,40.0,37.1
8870,1988.0,Michael Jordan*,2868.0,40.4,35.0
17743,2006.0,Kobe Bryant,2832.0,41.0,35.4
9748,1990.0,Michael Jordan*,2753.0,39.0,33.6
9298,1989.0,Michael Jordan*,2633.0,40.2,32.5
22439,2014.0,Kevin Durant,2593.0,38.5,32.0
10213,1991.0,Michael Jordan*,2580.0,37.0,31.5
24655,2017.0,Russell Westbrook,2558.0,34.6,31.6
6558,1982.0,George Gervin*,2551.0,35.7,32.3
11107,1993.0,Michael Jordan*,2541.0,39.3,32.6


In [7]:
#Creating a series from the CSV with new shooting percentages
player_initial_TS = player_data[['Year','Player','TS%', 'FGA/Game']]

#Converting the PTS column to a numberica dtype
player_initial_TS['TS%'] = pd.to_numeric(player_TS['TS%'])

# #Calculation, creation, mapping of new column called Points/Min
# FGA_per_game = player_data['FGA'] / player_data['G']
# player_TS['FGA/Game'] = FGA_per_game
player_initial_TS['FGA/Game'] = player_initial_TS['FGA/Game'].map('{:.1f}'.format)

#Sorting and displaying to check if the Minutes/Game column is calculating correctly
top_TS = player_initial_TS.sort_values(by='TS%', ascending=False)
top_TS.head(10)




NameError: name 'player_TS' is not defined

# Offensive Stat 2 - True Shooting Percentage

In [8]:
#Creating a series from the CSV with new shooting percentages
player_TS = player_data.loc[player_data['FGA/Game'] >= 10]
player_TS = player_TS[['Year','Player','TS%', 'FGA/Game']]

#Converting the PTS column to a numberica dtype
player_TS['TS%'] = pd.to_numeric(player_TS['TS%'])

#Sorting and displaying to check if the Minutes/Game column is calculating correctly
top_TS = player_TS.sort_values(by='TS%', ascending=False)
top_TS.head(10)

,Year,Player,TS%,FGA/Game
6559,1982.0,Artis Gilmore*,0.702,10.2
10102,1991.0,Johnny Dawkins,0.687,10.2
7661,1985.0,Artis Gilmore*,0.680,10.5
23634,2016.0,Stephen Curry,0.669,20.2
6939,1983.0,Artis Gilmore*,0.668,10.8
8700,1988.0,Charles Barkley*,0.665,16.0
6896,1983.0,Adrian Dantley*,0.661,18.3
9565,1990.0,Charles Barkley*,0.661,14.9
21339,2012.0,James Harden,0.660,10.1
8317,1987.0,Charles Barkley*,0.660,13.8


# Offensive Stat 3 - Effective Field Goal Percentage

# Offensive Stat 4 - Assits Per Minute Played

# Offensive Stat 5 - Offensive Rebound Per Minute Played